<a href="https://colab.research.google.com/github/quicksilverTrx/mining_of_massive_datasets/blob/main/colab_notebooks/Bloom_Filters_Colab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS246 - Colab 8
## Bloom Filters

### Setup

In this Colab we just need to install a [bloom_filter](https://github.com/hiway/python-bloom-filter), a Python library which offers an implementations of Bloom filters.  Run the cell below!

In [1]:
!pip install bloom_filter

### Data Loading

From the NLTK (Natural Language ToolKit) library, we import a large list of English dictionary words, commonly used by the very first spell-checking programs in Unix-like operating systems.

In [2]:
import nltk
nltk.download('words')

from nltk.corpus import words
word_list = words.words()
print(f'Dictionary length: {len(word_list)}')
print(word_list[:15])

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Dictionary length: 236736
['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron', 'Aaronic', 'Aaronical', 'Aaronite', 'Aaronitic', 'Aaru']


Then we load another dataset from the NLTK Corpora collection: ```movie_reviews```.

The movie reviews are categorized between *positive* and *negative*, so we construct a list of words (usually called **bag of words**) for each category.

In [3]:

nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
neg_reviews = []
pos_reviews = []

for fileid in movie_reviews.fileids('neg'):
  neg_reviews.extend(movie_reviews.words(fileid))
for fileid in movie_reviews.fileids('pos'):
  pos_reviews.extend(movie_reviews.words(fileid))

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


In [4]:
type(movie_reviews.words())

nltk.corpus.reader.util.ConcatenatedCorpusView

In [5]:
len(movie_reviews.words())

1583820

### Your task

In [6]:
# Displays frequency of 15 most common words in ‘movie_reviews’
nltk.FreqDist(movie_reviews.words()).most_common(15)

[(',', 77717),
 ('the', 76529),
 ('.', 65876),
 ('a', 38106),
 ('and', 35576),
 ('of', 34123),
 ('to', 31937),
 ("'", 30585),
 ('is', 25195),
 ('in', 21822),
 ('s', 18513),
 ('"', 17612),
 ('it', 16107),
 ('that', 15924),
 ('-', 15595)]

In [7]:
# all_words is a dictionary which contains the frequency of words in ‘movie_reviews’
all_words = nltk.FreqDist(movie_reviews.words())

In [8]:
nltk.FreqDist(movie_reviews.words())

FreqDist({'plot': 1513,
          ':': 3042,
          'two': 1911,
          'teen': 151,
          'couples': 27,
          'go': 1113,
          'to': 31937,
          'a': 38106,
          'church': 69,
          'party': 183,
          ',': 77717,
          'drink': 32,
          'and': 35576,
          'then': 1424,
          'drive': 105,
          '.': 65876,
          'they': 4825,
          'get': 1949,
          'into': 2623,
          'an': 5744,
          'accident': 104,
          'one': 5852,
          'of': 34123,
          'the': 76529,
          'guys': 268,
          'dies': 104,
          'but': 8634,
          'his': 9587,
          'girlfriend': 218,
          'continues': 88,
          'see': 1749,
          'him': 2633,
          'in': 21822,
          'her': 4522,
          'life': 1586,
          'has': 4719,
          'nightmares': 26,
          'what': 3322,
          "'": 30585,
          's': 18513,
          'deal': 219,
          '?': 3771,
          'wa

In [9]:
#unique words
len(all_words) 

39768

In this Colab, you will develop a very simplistic spell-checker.  By no means you should think of using it for a real-world use case, but it is an interesting exercise to highlight the strenghts and weaknesses of Bloom Filters!

In [24]:
from bloom_filter import BloomFilter

word_filter = BloomFilter(max_elements=2367368,error_rate=0.001)

for word in word_list:
  word_filter.add(word)

word_set = set(word_list)

If you executed the cell above, you now have 3 different variables in your scope:

1.   ```word_list```, a Python list containing the English dictionary (in case insensitive order)
2.   ```word_filter```, a Bloom filter where we have already added all the words in the English dictionary
3.   ```word_set```, a [Python set](https://docs.python.org/3.6/library/stdtypes.html#set-types-set-frozenset) built from the same list of words in the English dictionary

Let's inspect the size of each datastructure using the [getsizeof()](https://docs.python.org/3/library/sys.html#sys.getsizeof) method!



In [25]:
from sys import getsizeof

print(f'Size of word_list (in bytes): {getsizeof(word_list)}')
print(f'Size of word_filter (in bytes): {getsizeof(word_filter)}')
print(f'Size of word_set (in bytes): {getsizeof(word_set)}')
# YOUR CODE HERE


Size of word_list (in bytes): 2115960
Size of word_filter (in bytes): 64
Size of word_set (in bytes): 8388840


You should have noticed how efficient is the Bloom filter in terms of memory footprint!

Now let's find out how fast is the main operation for which we construct Bloom filters: *membership testing*. To do so, we will use the ```%timeit``` IPython magic command, which times the repeated execution of a single Python statement.

In [26]:
%timeit "California" in word_list
%timeit "California" in word_filter
%timeit "California" in word_set
# YOUR CODE HERE


1000 loops, best of 5: 473 µs per loop
100000 loops, best of 5: 18.6 µs per loop
The slowest run took 29.81 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 5: 58.1 ns per loop


Notice the performance gap between linear search on a list, multiple hash computations in a Bloom filter, and a single hash computation in a native Python ```Set()```.

We now have all the building blocks required to build our spell-checker, and we understand the performance tradeoffs of each datastructure we chose. Write a function that takes as arguments (1) a list of words, and (2) any of the 3 dictionary datastructures we constructed. The function must return the number of words which **do not appear** in the dictionary.

In [27]:
# YOUR CODE HERE
def spell_check(word_list,bloom_filter):
  l=0
  for word in word_list:
    if not word in bloom_filter:
      l=l+1

  return l 


In [28]:
spell_check(["bab"],word_filter)

1

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!